In [1]:
# Set some notebook defaults
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20;

# Display information about the environment
VERSION, Base.Threads.nthreads(), Base.Sys.CPU_THREADS

(v"1.5.3", 8, 8)

In [2]:
using Random, Distributions, BenchmarkTools, Tullio, LoopVectorization, DataFrames, StaticKernels, SantasLittleHelpers

In [3]:
N = 10^8
d = Normal()
df = DataFrame(:data => rand(d, N))

,data
,Float64
1,1.01386
2,-0.0648729
3,-0.998316
4,0.149736
5,-0.642035
6,0.480422
7,-0.0102204
8,1.12168
9,0.586801


In [4]:
# Calculate sma using StaticKernels and SantasLittleHelpers
# Contains code originally from 

k = makekernel(mean,-9:0)
df[!, "sma_sk"] = applyrolling(k, df.data)
df

,data,sma_sk
,Float64,Float64
1,1.01386,1.01386
2,-0.0648729,0.474494
3,-0.998316,-0.0164429
4,0.149736,0.0251018
5,-0.642035,-0.108325
6,0.480422,-0.0102009
7,-0.0102204,-0.0102037
8,1.12168,0.131282
9,0.586801,0.181896


In [5]:
# Contains code originally from Michael Abbott

tmp = Array{Union{Missing, Float64}}(missing, length(df.data));

function f_tullio(data, sma)
    out_view = @view sma[10:end]
    @tullio out_view[i] = 0.1*data[i+o]  o in 0:9  # verbose=true;
    sma
end

df[!, "sma_tullio"] = f_tullio(df.data, tmp)
df

,data,sma_sk,sma_tullio
,Float64,Float64,Float64?
1,1.01386,1.01386,missing
2,-0.0648729,0.474494,missing
3,-0.998316,-0.0164429,missing
4,0.149736,0.0251018,missing
5,-0.642035,-0.108325,missing
6,0.480422,-0.0102009,missing
7,-0.0102204,-0.0102037,missing
8,1.12168,0.131282,missing
9,0.586801,0.181896,missing


In [6]:
# Contains code originally from Peter Deffebach

out = Vector{Union{Missing, Float64}}(undef, N)

function rollingmean2(x, m, out)
    N = length(x)
    out[1:(m-1)] .= missing
    sm = mean(@view x[1:m])
    out[m] = sm
    @inbounds @simd for i in (m+1):length(x)
        sm += (x[i] - x[i-m]) / m
        out[i] = sm
    end
    return out 
end

rollingmean2 (generic function with 1 method)

In [7]:
df[!, "sma_simd"] = rollingmean2(df.data, 10, out)
df

,data,sma_sk,sma_tullio,sma_simd
,Float64,Float64,Float64?,Float64?
1,1.01386,1.01386,missing,missing
2,-0.0648729,0.474494,missing,missing
3,-0.998316,-0.0164429,missing,missing
4,0.149736,0.0251018,missing,missing
5,-0.642035,-0.108325,missing,missing
6,0.480422,-0.0102009,missing,missing
7,-0.0102204,-0.0102037,missing,missing
8,1.12168,0.131282,missing,missing
9,0.586801,0.181896,missing,missing


In [8]:
df.sma_sk[10:end] ≈ df.sma_tullio[10:end]

true

In [9]:
df.sma_sk[10:end] ≈ df.sma_simd[10:end]

true

In [10]:
@benchmark applyrolling(k, df.data)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     999.771 ms (0.09% GC)
  median time:      1.076 s (0.13% GC)
  mean time:        1.065 s (4.00% GC)
  maximum time:     1.140 s (7.71% GC)
  --------------
  samples:          5
  evals/sample:     1

In [11]:
@benchmark f_tullio(df.data, tmp)

BenchmarkTools.Trial: 
  memory estimate:  7.61 KiB
  allocs estimate:  115
  --------------
  minimum time:     136.106 ms (0.00% GC)
  median time:      144.978 ms (0.00% GC)
  mean time:        145.947 ms (0.00% GC)
  maximum time:     164.684 ms (0.00% GC)
  --------------
  samples:          35
  evals/sample:     1

In [12]:
@benchmark rollingmean2(df.data, 10, out)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     125.207 ms (0.00% GC)
  median time:      134.323 ms (0.00% GC)
  mean time:        138.642 ms (0.00% GC)
  maximum time:     162.930 ms (0.00% GC)
  --------------
  samples:          37
  evals/sample:     1